In [1]:
!pip install langchain langchain_community language-tool-python langchain-ollama jq langchain_huggingface langchain_chroma faiss-cpu

In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
import csv
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import JSONLoader
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
import json
from langchain_chroma import Chroma

# 문서 로드

In [3]:
def load_example_sentences(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return [Document(page_content=sentence) for sentence in data]

example_sentences = load_example_sentences(r"C:\Users\user\Desktop\eng-word\utils\json_words\combined\combined_examples.json")

example_sentences

[Document(page_content='Let me introduce my aunt.'),
 Document(page_content='I am going to play soccer.'),
 Document(page_content='The boy is my brother.'),
 Document(page_content='Jimmy is going to exercise every day.'),
 Document(page_content='The girl is my sister.'),
 Document(page_content='Let me introduce my cousin to you.'),
 Document(page_content='We are going to learn Korean history.'),
 Document(page_content='I’m going to run a marathon.'),
 Document(page_content='He is going to visit his grandpa.'),
 Document(page_content='The woman is my mom.'),
 Document(page_content='He set a new world record for swimming.'),
 Document(page_content='Just sit down and relax for an hour.'),
 Document(page_content='He repairs computers.'),
 Document(page_content='The teacher repeated the question.'),
 Document(page_content='The doctor reported that the patient recovered.'),
 Document(page_content='The firefighters rescued children from the burning building.'),
 Document(page_content='You wil

# 모델 생성

In [4]:
# llm = ChatOllama(model="Lama3.2-korean:latest", max_token = 500)
llm = ChatOllama(model="llama3.2", max_token=50, temperature=0)

# 프롬프트 

In [5]:
# PromptTemplate 설정
# prompt_template = PromptTemplate(
#     input_variables=["word"],
#     template="""
#     Write a simple English sentence (5-8 words) that an elementary school student can understand. 
#     The sentence must include the word '{word}' exactly as it is, without any modifications or variations.
#     Avoid using conjunctions and ensure that the sentence is not a compound sentence. 
#     Make sure to create only one complete sentence for each word. Use words commonly used in the US.
#     No describe, only sentence
#     """,
# )


# # 프롬프트 템플릿 정의
# prompt_template = PromptTemplate(
#     input_variables=["query"],
#     template="""
    
#     #QUERY
#     Write a simple English sentence (5-8 words) that an elementary school student can understand.
#     The sentence must include the word '{query}' exactly as it is, without any modifications or variations.
#     Avoid using conjunctions and ensure that the sentence is not a compound sentence.
#     Make sure to create only one complete sentence for each word. Use words commonly used in the US.
    
#     #IMPORTANCE
#     Create only one complete sentence.
#     """,
# )

# prompt_template = PromptTemplate(
#     input_variables=["query"],
#     template="""
    
#     #QUERY
#     Write one simple, complete English sentence (5-8 words) that an elementary school student can understand.
#     The sentence must include the word '{query}' exactly as it is, without any modifications, variations, or additional forms.
#     Use only a single statement, not a question or command, and avoid conjunctions, question forms, or compound sentences.
    
#     #IMPORTANCE
#     Ensure that only one sentence is created and that it contains the word '{query}' precisely as specified.
#     Use only words commonly used in the US.
#     """,
# )

# prompt_template = PromptTemplate(
#     input_variables=["query"],
#     template="""
#     You are an assistant that must follow strict guidelines without exception.

#     # QUERY
#     Write exactly *one* simple, complete English sentence (5-8 words) that an elementary school student can understand.
#     The sentence must include the word '{query}' exactly as it is, without any modifications, variations, or additional forms.

#     # RULES
#     1. Use only a single statement, not a question or command.
#     2. Do not use conjunctions, question forms, or compound sentences.
#     3. Ensure that *only one* sentence is created.
#     4. The word '{query}' must be included *exactly as specified* (no changes, no variations).
#     5. Use only words commonly used in the US.

#     # IMPORTANCE
#     If you are unable to generate the exact sentence following these rules, do not generate anything at all.
#     """
# )

prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
    You must create exactly one simple, complete English sentence that an elementary school student can understand.
    
    - The sentence must include the word '{query}' exactly as specified, without any modifications or additional forms.
    - Use only a single statement, not a question, command, or compound sentence.
    - The sentence should be between 5 and 8 words long.
    - Do not create multiple sentences or add any additional information.
    - Use only common, everyday words that are simple for an elementary school student in the US to understand.

    If you cannot create a sentence following these rules exactly, respond with "Unable to create a valid sentence."
    """
)


# 임베딩 생성

In [6]:
# embeddings = OpenAIEmbeddings()
model_name = "sentence-transformers/all-MiniLM-L6-v2"


# model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu","trust_remote_code":True},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

c:\Users\user\anaconda3\envs\eng\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# vectoc store

In [7]:
# 벡터스토어 저장
vector_store = FAISS.from_documents(example_sentences, hf_embeddings)
# 생성된 벡터 스토어를 디스크에 저장
vector_store.save_local("./sentence_vectorstore")

# 저장할 경로 지정
DB_PATH = "./sentence_vectorstore"

# # DB 생성
# db = Chroma.from_documents(
#     documents=example_sentences, embedding=hf_embeddings, collection_name="sentence_db",persist_directory=DB_PATH
# )

In [8]:
# # 디스크에서 문서를 로드합니다.
# DB_PATH = "./sentence_vectorstore"

# chroma_db = Chroma(
#     persist_directory=DB_PATH,
#     embedding_function=hf_embeddings,
#     collection_name="sentence_db",
# )

In [9]:
# 저장된 벡터 스토어 로드
loaded_vector_store = FAISS.load_local(
    "./sentence_vectorstore",
    hf_embeddings,
    allow_dangerous_deserialization=True  # 신뢰된 파일에서만 사용
)

# 검색기 생성

In [10]:
retriever = loaded_vector_store.as_retriever()
# retriever = chroma_db.as_retriever()


In [11]:
retriever.invoke("eras")

[Document(page_content='Do you remember the members?'),
 Document(page_content='We have a history class today.'),
 Document(page_content='Those are his albums.'),
 Document(page_content='We will never forget it.')]

# 출력 파서

In [12]:
# output_parser = CommaSeparatedListOutputParser()
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# 체인 생성

In [13]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template, "document_variable_name": "query"},
)

In [14]:
# qa_chain.invoke({"query":"eraser"})

In [15]:
# chain = prompt_template | qa_chain | output_parser
# chain = prompt_template | qa_chain


# 실행 코드(테스트)

In [16]:
# JSON 파일에서 단어 목록 불러오기
with open(r"C:\Users\user\Desktop\eng-word\utils\json_words\combined_words.json", "r", encoding="utf-8") as file:
    words = json.load(file)

In [18]:
import csv
from tqdm import tqdm

# CSV 파일 생성
sentences = []

slice_word = words[:10]

# CSV 파일 생성
with open("grammar_sentences.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["문장 번호", "문장", "포함된 단어"])

    # 각 단어에 대해 문장 생성 및 저장
    for idx, word in tqdm(enumerate(words, start=1)):
        response = qa_chain.invoke({"query": word})  # 체인 실행

        # response의 형식이 예상대로 되어 있는지 확인
        if 'result' in response:
            generated_sentence = response['result']

            # 문장에 포함된 단어가 정확히 있는지 검증
            if word in generated_sentence:
                writer.writerow([idx, generated_sentence, word])
                print(generated_sentence, word)
            else:
                print(f"생성된 문장이 규칙을 따르지 않습니다. 단어: {word}")
        else:
            print(f"응답 형식이 잘못되었습니다. 단어: {word}")


1it [00:02,  2.87s/it]

This is my aunt. aunt


2it [00:05,  2.85s/it]

생성된 문장이 규칙을 따르지 않습니다. 단어: be going to


3it [00:08,  2.83s/it]

...

Let's take a taxi at the small airport. taxi


2171it [1:41:20,  2.83s/it]

The train station is on the same side. train


2172it [1:41:23,  2.80s/it]

The truck is dumping the trash. truck


In [16]:
from tqdm import tqdm
# CSV 파일 생성
sentences = []

slice_word = words[:10]

# CSV 파일 생성
with open("grammar_sentences.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["문장 번호", "문장", "포함된 단어"])

    # 각 단어에 대해 문장 생성 및 저장
    for idx, word in tqdm(enumerate(words, start=1)):
        response = qa_chain.invoke({"query": word})  # 체인 실행
        
        # # 파싱된 문장 기록
        # for sentence in response:
        #     writer.writerow([idx, sentence, word])
        #     print(sentence)
        writer.writerow([idx, response['result'], word])
        print(response['result'], word)
        

1it [00:06,  6.68s/it]

I'd like to introduce my aunt. aunt


2it [00:09,  4.44s/it]

I'm going to the concert hall. be going to


3it [00:14,  4.48s/it]

...

He is Mark. mark


136it [07:21,  2.88s/it]

I can raise the letters. raise


136it [07:24,  3.27s/it]


KeyboardInterrupt: 